In [43]:
import nltk
import re
import string
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from nltk.util import pr
from nltk.corpus import stopwords

In [10]:
df=pd.read_csv('twitter.csv')
print(df)
print(df.columns)

       Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                                   tweet  
0      !!! RT @m

In [60]:
df["labels"]=df['class'].map({0:"Hate Speech", 1:"Offensive Language", 2:"Neutral Language"})
#print(df['labels'])
print(df['labels'].value_counts)

<bound method IndexOpsMixin.value_counts of 0          Neutral Language
1        Offensive Language
2        Offensive Language
3        Offensive Language
4        Offensive Language
                ...        
24778    Offensive Language
24779      Neutral Language
24780    Offensive Language
24781    Offensive Language
24782      Neutral Language
Name: labels, Length: 24783, dtype: object>


In [19]:
data=df[['tweet','labels']]
data.head(5)

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,Neutral Language
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Language
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Language
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Language
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Language


Cleaning The Dataset

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [34]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [39]:
stemmer=nltk.SnowballStemmer('english')

In [41]:
def clean(text):
    text=str(text).lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<."?>+','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    text=[word for word in text.split(' ') if word not in stopwords.words('english')]
    text=' '.join(text)
    text=[stemmer.stem(word) for word in text.split(' ')]
    text=' '.join(text)
    return text

data['tweet']=data['tweet'].apply(clean)

<ipython-input-41-0d2c38a3319e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet']=data['tweet'].apply(clean)


In [58]:
data['tweet']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [45]:
x=np.array(data['tweet'])
y=np.array(data['labels'])

cv=CountVectorizer()
x=cv.fit_transform(x)
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=42)

clf=DecisionTreeClassifier()
clf.fit(x_train,y_train)
clf.score(x_test,y_test)
y_pred=clf.predict(x_test)

In [52]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the model: {accuracy}")


Accuracy of the model: 0.8700827113173291


Dataset Validation

In [54]:
sample=""

data=cv.transform([sample]).toarray()
print(clf.predict(data))

['Neutral Language']


In [55]:
import pickle


filename_cv = 'vectorizer.pkl'
pickle.dump(cv, open(filename_cv, 'wb'))

filename_model = 'model.pkl'
pickle.dump(clf, open(filename_model, 'wb'))

print(f"Model saved as {filename_model}")
print(f"Vectorizer saved as {filename_cv}")

Model saved as model.pkl
Vectorizer saved as vectorizer.pkl
